<a href="https://colab.research.google.com/github/MichalSlowakiewicz/Machine-Learning/blob/master/Homework11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Understanding Deconvolution in Autoencoders**
---------------

In class, we worked with autoencoders built from multilayer perceptrons (MLPs). However, encoders are often constructed using convolutional architectures to better capture spatial patterns. In this assignment, you'll explore how the decoder can use deconvolutional (transposed convolution) layers to reverse and mirror the operations performed by the convolutional encoder.

While convolutional encoders are relatively well understood, **decoding (or upsampling) the compressed representation** using **deconvolutional layers** (also known as **transposed convolutions**) often raises questions.

This assignment is particularly relevant because deconvolution is a core component of the U-Net architecture, a prominent neural network used extensively in image segmentation tasks.

Your main objective is to deeply understand **how transposed convolution layers work**, and explain them in both words and visuals.


## **The Objective**

Understand and clearly explain how **transposed convolutions** work. Use 2D transposed convolutions and a small grid of 2D points as a working example.

You may need to do some additional reading to complete this assignment.

## **Tasks & Deliverables**

### 1. **Theory Exploration**

Using markdown cells in your Colab notebook, answer the following:

- What is a **transposed convolution**?
- How does it differ from a regular convolution?
- How does it upsample feature maps?
- What are **stride**, **padding**, and **kernel size**, and how do they influence the result in a transposed convolution?
- To earn full two points, your explanation must be detailed enough for a reader to reproduce the upsampling process step by step.


### 2. **Manual Diagram (by your hand, not a generated image)**

Carefully plan and draw **by hand** a diagram or a set of diagrams that:

- Explain the process of using **transposed convolution**.
- Use an example of a **small input grid of 2D points** which gets expanded into a larger output grid.
- Explain how stride, padding, and the kernel shape affect the result.
- Show intermediate steps of the operation, not just input and output.

**Scan or photograph your diagram(s)**, and upload it to your **GitHub repository** for this course.

Then embed it in your Colab notebook using markdown (you can find examples on *how to do it* in previous notebooks related to this class, e.g. the one on linear regression or the one on the MLP network).


### 3. **Publish on GitHub**  
   - Place the Colab notebook in your **GitHub repository** for this course.
   - In your repository’s **README**, add a **link** to the notebook and also include an **“Open in Colab”** badge at the top of the notebook so it can be launched directly from GitHub.


# **Theory Exploration**

**What is a transposed convolution?**

**How does it differ from a regular convolution?**

  "Transponowana konwolucja" to operacja, która zwiększa rozmiar otrzymanej mapy cech (nie wiem czy mapa cech to odpowiednie określenie po polsku, ang. *featurre map*), tzn. generuje output większy niż otrzymany wcześniej input (np. na podstawie mapy 2x2 tworzy mapę 4x4). "Transponowana konwolucja" intuicyjnie działą jak konwolucja tylko odwrotnie - zamiast  zmniejszać obraz, powiększa go używając filtra (*kernel*). Warto w tym miejscu wspomnieć, w jaki sposób kontrolujemy jak bardzo *output* będzie większy niż *input* i generalnie, jakie parametry musimy ustawić przed rozpoczęcziem tej procedury. Wpływają na to parametry: *stride* - im większy będzie, tym bardziej obraz z *inputu* zostanie powiększony; *padding* - dodaje zera wokół obrazu, co sprawia, iż kontrolujemy ramkę ?; *size of kernel* - po prostu jest to rozmiar filtra (*kernela*) działającego analogicznie jak dla konwolucji, im większy będzie kernel tym bardziej rozmyte, ale płynniejszzy, będzie *output*.